# Parse analyst names from Capital IQ and IBES datasets and reformat to prepare to merge on analyst name

In [1]:
import numpy as np, re, string
import conversions as cv # Custom function to read and format sas files into pandas dataframe

In [2]:
def get_names_ciq(s,last=True):
    delim = re.compile(r'( of )|(;)')
    rmv = re.compile(r'[\(?\)-]|III|II|IV|Jr\.|Sr\.|,.*|from.*|with.*|–.*| - .*')
    rmv2 = re.compile(r'Analyst (for|with)')
    dblnms = re.compile(r'(Van Den|Van|[dD][ei]n?)\s+\w*')
    s = re.sub(r'\d+', '', str(s)) 
    if str(s).strip().endswith("."):
        s = re.sub(r'\.',"",s)
    else:
        s = s
    nm = re.sub(rmv,"",str(s))
    nm2 = re.split(delim,nm)[0].split(" for ")[-1]
    try:
        if last:
            if re.search(dblnms,nm2):
                ln = re.search(dblnms,nm2).group()
            else:
                if re.search(r'\.',nm2):
                    nm2 = nm2.split(".")[-1]
                else:
                    nm2 = nm2.translate(str.maketrans('', '', string.punctuation))
                ln = nm2.split()[-1]
            ln = re.sub("\W","",ln)
            ln = ln.encode("ascii", errors="ignore").decode()
            return ln.upper()
        else:
            fn = nm2.split()[0]
            fn = re.sub("\W","",fn)
            fn = fn.encode("ascii", errors="ignore").decode()
            return fn.upper()
    except: return "Error"
    
def get_names_ibes(s):
    rmv = re.compile(r'(,|III|II|IV|JR|SR|\s{3,}\w).*')
    if len(s.split("/")) > 1:
        strs = s.split("/")
        s1 = re.sub(rmv,"",strs[0])
        s2 = re.sub(rmv,"",strs[1])
        return s1.upper(),s2.upper()
    else:
        if len(s.split()) == 1: s = s
        else: s = s[:-1].strip()
        s = re.sub(rmv,"",s)
        s = re.sub("\W","",s)
        return s.upper(),""


In [3]:
filePath = r"G:\\My Drive\\Research\\FLP_Analyst_Soft_Skills\\FLP_Analyst_Skill_Shared\\Data\\capital IQ\\Adjusted"
ciqFileName = filePath + r"\\ciqanalystfirmyear_20240330.sas7bdat"
ibesFileName = filePath + r"\\ibesanalystfirmyear_20240330.sas7bdat"

In [4]:
ciq = cv.readsas(ciqFileName)
ibes = cv.readsas(ibesFileName)
print("# of CIQ Analyst-Gvkey-Year obs:",len(ciq))
print("# of IBES Analyst-Gvkey-Year obs:",len(ibes))

Used latin-1 to decode G:\\My Drive\\Research\\FLP_Analyst_Soft_Skills\\FLP_Analyst_Skill_Shared\\Data\\capital IQ\\Adjusted\\ciqanalystfirmyear_20240330.sas7bdat
Used UTF-8 to decode G:\\My Drive\\Research\\FLP_Analyst_Soft_Skills\\FLP_Analyst_Skill_Shared\\Data\\capital IQ\\Adjusted\\ibesanalystfirmyear_20240330.sas7bdat
# of CIQ Analyst-Gvkey-Year obs: 1065634
# of IBES Analyst-Gvkey-Year obs: 972595


In [5]:
ciq[ciq.companyofperson == "Crédit Suisse AG, Research Division"]

,transcriptpersonid,proid,companyid,gvkey,year,transcriptpersonname,companyofperson
63,100035.0,29154009.0,1.681540e+05,001004,2015.0,Robert Spingarn,"Crédit Suisse AG, Research Division"
64,100035.0,29154009.0,1.681540e+05,001004,2016.0,Robert Spingarn,"Crédit Suisse AG, Research Division"
65,100035.0,29154009.0,1.681540e+05,001004,2017.0,Robert Spingarn,"Crédit Suisse AG, Research Division"
66,100035.0,29154009.0,1.681540e+05,001004,2018.0,Robert Spingarn,"Crédit Suisse AG, Research Division"
67,100035.0,29154009.0,1.681540e+05,001004,2019.0,Robert Spingarn,"Crédit Suisse AG, Research Division"
...,...,...,...,...,...,...,...
1065044,312280.0,311968276.0,2.921972e+08,352853,2023.0,Varun Ahuja,"Crédit Suisse AG, Research Division"
1065148,158823.0,29090300.0,1.774496e+09,353070,2022.0,Larry Gandler,"Crédit Suisse AG, Research Division"
1065179,299086.0,278541573.0,2.087483e+08,353138,2022.0,Lokesh Garg,"Crédit Suisse AG, Research Division"
1065270,349784.0,533359547.0,1.671691e+09,353444,2022.0,Mirza Faham Baig,"Crédit Suisse AG, Research Division"


Validate function to properly format names - extract only the last name and make it all capitalized to match analysts' last name in IBES.

In [6]:
ciq_nms = ['Alex Brand','(Actavian Popescu?)','Adam B. Frisch','Analyst for Bev Dijinson',
           'Bonnie K.Wachtel','Brooks O’Neil','John','Mark R. Gulley','P. J. Juvekar',
           'Patrick J. Forkin III','K.C. Ambrecht','Vern Essi, Jr.','Judson\xa0E.\xa0Bailey',
          'E. Russell McAllister from Merriman','Joseph B. Nadol, III','Eric J. Grubelich, CFA',
          'Robert McCarthy, Jr.', 'J. Van Rhee','C. Todd Maiden','John Yurning(?)','Kim De Paoli',
           'Morton Oâ€™Tool Smith',"Shannon O'Callaghan","Daniel Oâ€™Sullivan","Allan Gould for Jeff Shelton",
          "Oscar Cabrera with Goldman Sachs","Irit Elrad-Jakoby","Erika Penala -","William Di Tullio","Operator.	",
           "Saravanan V.N.","Dmitry Trembovolsky – Goldman Sachs Group Inc....	","Spencer Wang – Credit Suisse	",
          "[ph] Sutanto Widjaja (58:02)	","Jason Late - Ares Management Limited; Managing...	",
           "Robert Van Den Broek","Jason L.;Loews Corporation;Analyst"]
print([get_names_ciq(s,last=True) for s in ciq_nms])

['BRAND', 'POPESCU', 'FRISCH', 'DIJINSON', 'WACHTEL', 'ONEIL', 'JOHN', 'GULLEY', 'JUVEKAR', 'FORKIN', 'AMBRECHT', 'ESSI', 'BAILEY', 'MCALLISTER', 'NADOL', 'GRUBELICH', 'MCCARTHY', 'VANRHEE', 'MAIDEN', 'YURNING', 'DEPAOLI', 'SMITH', 'OCALLAGHAN', 'OSULLIVAN', 'SHELTON', 'CABRERA', 'ELRADJAKOBY', 'PENALA', 'DITULLIO', 'OPERATOR', 'VN', 'TREMBOVOLSKY', 'WANG', 'WIDJAJA', 'LATE', 'VANDENBROEK', 'Error']


In [7]:
ibes_nms = ["PRICE/PRICE   J","GARCIA-CANTER J","DE VERTEUIL   I","DEATHERAGE/MASSICK","BOSSHARD      E",
      "MCALENNEY,JR. E","WELLS III     H","NICKLIN","LEISURE-TOURI","HUNTER BYRNE  M","WOLFENBERGER  M",
      "VAN DEN BROEK R","YOUNG/C. BARN","O'BRIEN       K","WARNER-WILKE  C","MOUSSATOS, PH.D.   L"]
print([get_names_ibes(s) for s in ibes_nms])

[('PRICE', 'PRICE'), ('GARCIACANTER', ''), ('DEVERTEUIL', ''), ('DEATHERAGE', 'MASSICK'), ('BOSSHARD', ''), ('MCALENNEY', ''), ('WELLS', ''), ('NICKLIN', ''), ('LEISURETOURI', ''), ('HUNTERBYRNE', ''), ('WOLFENBERGER', ''), ('VANDENBROEK', ''), ('YOUNG', 'C. BARN'), ('OBRIEN', ''), ('WARNERWILKE', ''), ('MOUSSATOS', '')]


Reformat analysts' names to prepare for match between Capital IQ and IBES

In [8]:
ciq['fname'] = ciq.transcriptpersonname.apply(lambda x: get_names_ciq(x,last=False))
ciq['lname'] = ciq.transcriptpersonname.apply(get_names_ciq)
ibes['lname1'] = ibes.analyst.apply(lambda x: get_names_ibes(x)[0])
ibes['lname2'] = ibes.analyst.apply(lambda x: get_names_ibes(x)[1])

In [9]:
ciq2 = ciq[['transcriptpersonid','proid','companyofperson','year','gvkey','fname','lname']]
ibes2 = ibes[['ticker','estimid','amaskcd','year','permno','gvkey','analyst','lname1','lname2']]

In [10]:
ciq2[ciq2.companyofperson == "Crédit Suisse AG, Research Division"]

,transcriptpersonid,proid,companyofperson,year,gvkey,fname,lname
63,100035.0,29154009.0,"Crédit Suisse AG, Research Division",2015.0,001004,ROBERT,SPINGARN
64,100035.0,29154009.0,"Crédit Suisse AG, Research Division",2016.0,001004,ROBERT,SPINGARN
65,100035.0,29154009.0,"Crédit Suisse AG, Research Division",2017.0,001004,ROBERT,SPINGARN
66,100035.0,29154009.0,"Crédit Suisse AG, Research Division",2018.0,001004,ROBERT,SPINGARN
67,100035.0,29154009.0,"Crédit Suisse AG, Research Division",2019.0,001004,ROBERT,SPINGARN
...,...,...,...,...,...,...,...
1065044,312280.0,311968276.0,"Crédit Suisse AG, Research Division",2023.0,352853,VARUN,AHUJA
1065148,158823.0,29090300.0,"Crédit Suisse AG, Research Division",2022.0,353070,LARRY,GANDLER
1065179,299086.0,278541573.0,"Crédit Suisse AG, Research Division",2022.0,353138,LOKESH,GARG
1065270,349784.0,533359547.0,"Crédit Suisse AG, Research Division",2022.0,353444,MIRZA,BAIG


In [11]:
ciq2.loc[ciq2.companyofperson.str.contains('\u2019')==True,]

,transcriptpersonid,proid,companyofperson,year,gvkey,fname,lname


Save down files with converted name format

In [12]:
ciq2.to_csv(filePath+r"\\ciqAFY_FmtdNms_20240330.csv",index=False,encoding='utf8')
ibes2.to_csv(filePath+r"\\ibesAFY_new_vint_FmtdNms_20240330.csv",index=False,encoding='Latin-1')

Print variable lengths for merge back to SAS.

In [13]:
measurer = np.vectorize(len)
ciqLen = measurer(ciq2.values.astype(str)).max(axis=0)
ibesLen = measurer(ibes2.values.astype(str)).max(axis=0)

In [14]:
print("CIQ")
for l,col in zip(ciqLen,ciq2.columns):
    print(col,": ",l)
    
print("\nIBES")
for l,col in zip(ibesLen,ibes2.columns):
    print(col,": ",l)
#print("ciqLen: ",)

CIQ
transcriptpersonid :  8
proid :  12
companyofperson :  92
year :  6
gvkey :  6
fname :  19
lname :  22

IBES
ticker :  6
estimid :  8
amaskcd :  8
year :  6
permno :  7
gvkey :  6
analyst :  20
lname1 :  17
lname2 :  12


In [15]:
len(ciq2[ciq2.lname=="Error"])

5